In [3]:
import pandas as pd

df = pd.read_csv('fil_data_movies_lessthan10k_removed.csv')
df = df.drop(columns=['id', 'title','release_date',  'adult','original_title', 'backdrop_path', 'homepage', 'imdb_id', 'overview', 'poster_path', 'tagline','spoken_languages' ])
#df.head()

In [2]:
df_encoded = pd.get_dummies(df['genres'].str.split(', ').explode()).groupby(level=0).sum()
#print(df_encoded.head())

df_encoded2 = pd.get_dummies(df['original_language'].str.split(', ').explode()).groupby(level=0).sum()
df_new = pd.DataFrame()
df_new['en'] = df_encoded2['en']
df_new['other'] = df_encoded2.drop('en', axis=1).sum(axis=1)
#print(df_new)

df_encoded3 = pd.get_dummies(df['production_companies'].str.split(', ').explode()).groupby(level=0).sum()
mask = (df_encoded3.sum() >= 180)
df_encoded3 = df_encoded3.loc[:, mask.values]
#print(df_encoded3.sum())

df_encoded4 = pd.get_dummies(df['production_countries'].str.split(', ').explode()).groupby(level=0).sum()
df_new2 = pd.DataFrame()
df_new2['United States of America'] = df_encoded4['United States of America']
df_new2['United Kingdom'] = df_encoded4['United Kingdom']
df_new2['other2'] = df_encoded4.drop(['United States of America', 'United Kingdom'], axis=1).sum(axis=1)
#print(df_new2.sum())


In [4]:
df = pd.concat([df, df_encoded], axis=1)
df = df.drop(columns=['genres'])

df = pd.concat([df, df_new], axis=1)
df = df.drop(columns=['original_language'])

df = pd.concat([df, df_encoded3], axis=1)
df = df.drop(columns=['production_companies'])

df = pd.concat([df, df_new2], axis=1)
df = df.drop(columns=['production_countries'])

#df.head()

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Define the features and the target
X = df.drop('vote_average', axis=1)
y = df['vote_average']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)


In [5]:
# Import metrics
from sklearn import metrics

# Make predictions on the entire data set
y_pred_all = model.predict(X_test)

# Compare the predicted values with the actual values
df_scores_comp_all = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_all})

# Print the comparison dataframe
#print(df_scores_comp_all)

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Create a list of models
models = [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor()]

# Define the hyperparameters for each model
hyperparameters = {
    'LinearRegression': {},
    'DecisionTreeRegressor': {'max_depth': [None, 5, 10]},
    'RandomForestRegressor': {'n_estimators': [100, 200, 300], 'max_depth': [None, 5, 10]}
}

for model in models:
    model_name = model.__class__.__name__
    
    # Initialize the grid search
    grid_search = GridSearchCV(model, hyperparameters[model_name])
    
    # Train the model with the grid search
    grid_search.fit(X_train, y_train)
    
    # Make predictions with the best found model
    y_pred = grid_search.best_estimator_.predict(X_test)
    
    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    r2_percentage = r2 * 100
    #print(f"Model: {model_name}, Best Hyperparameters: {grid_search.best_params_}, R-squared (%): {r2_percentage}")



NameError: name 'X_train' is not defined

In [7]:
from sklearn.ensemble import RandomForestRegressor

best_params_ = grid_search.best_params_
model = RandomForestRegressor(**best_params_)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(r2_score(y_test, y_pred)*100)

53.838018162441195
